# Human vs AI Flappy Bird gameplay analysis

## Imports

In [1]:
import random
import time
import json
import base64
import csv
from google.colab import files
import datetime
from zoneinfo import ZoneInfo
import zipfile

## Flappy Bird no-GUI game engine

### Game-specific constants

In [19]:
GAME_WIDTH = 500
GAME_HEIGHT = 700

FIXED_DT = 0.0167

MAX_EPISODES = 30
MAX_FRAMES = 30000

### Classes

#### Bird class

In [3]:
class Bird:
  def __init__(self, game):
    self.game = game

    self.isFlapping = False
    self.flapCooldown = 0

    self.width = self.game.BIRD_WIDTH
    self.height = self.game.BIRD_HEIGHT

    self.x = self.game.GAME_WIDTH * self.game.BIRD_X_RATIO
    self._y = (self.game.GAME_HEIGHT - self.height) / 2

    self.velocity = 0

  @property
  def y(self):
    return self._y;
  @y.setter
  def y(self, v):
    if (v > self.game.GAME_HEIGHT or v < 0 - self.height):
      self.game.gameIsOver = True
    else:
     self._y = v;

  @property
  def hitbox(self):
    return {
      'x': self.x,
      'y': self.y,
      'width': self.width,
      'height': self.height,
    }

  def update(self, dt):
    self.flapCooldown = max(0, self.flapCooldown - dt)
    self.fall(dt)

  def reset(self):
    self.x = self.game.GAME_WIDTH * self.game.BIRD_X_RATIO
    self._y = (self.game.GAME_HEIGHT - self.height) / 2

    self.velocity = 0

  def fall(self, dt):
    self.velocity += self.game.GRAVITY * dt
    self.y += self.velocity * dt

  def flapWings(self):
    self.velocity = -300


#### Pipes class

In [4]:
class Pipes:
  def __init__(self, game):
    self.game = game

    self.gap = self.game.PIPE_GAP

    self._x = self.game.PIPE_START_POINT
    self.topY = random.random() * (self.game.GAME_HEIGHT - self.gap)
    self.botY = self.topY + self.gap

    self.width = self.game.PIPE_WIDTH
    self.topHeight = self.topY
    self.botHeight = self.game.GAME_HEIGHT - self.botY

    self.velocity = self.game.PIPE_VELOCITY

    self.passed = False

  @property
  def x(self):
    return self._x
  @x.setter
  def x(self, v):
    if (v + self.width < 0):
      self.game.removePipePair(self)
    else:
      self._x = v

  @property
  def topHitbox(self):
    return {
      'x': self.x,
      'y': 0,
      'width': self.width,
      'height': self.topHeight,
    }
  @property
  def botHitbox(self):
    return {
      'x': self.x,
      'y': self.botY,
      'width': self.width,
      'height': self.botHeight,
    }

  def update(self, dt):
    self.x -= self.velocity * dt

  def reset(self):
    self.x = self.game.PIPE_START_POINT
    self.topY = random.random() * (self.game.GAME_HEIGHT - self.gap)
    self.botY = self.topY + self.gap

    self.topHeight = self.topY
    self.botHeight = self.game.GAME_HEIGHT - self.botY


#### Game class

In [35]:
class Game:
  def __init__(self, GAME_WIDTH, GAME_HEIGHT, record=False):
    self.record = record
    self.frameStates = [] if record else None
    self.episodeId = 0
    self.frameId = 0

    self.genome = None

    self.AGENT_TYPE = "human"
    self.GAME_WIDTH = GAME_WIDTH
    self.GAME_HEIGHT = GAME_HEIGHT
    self.BIRD_X_RATIO = 0.3
    self.BIRD_WIDTH = 50
    self.BIRD_HEIGHT = 40
    self.GRAVITY = 600
    self.PIPE_START_POINT = 500
    self.PIPE_GAP = 200
    self.PIPE_WIDTH = 75
    self.PIPE_VELOCITY = 100
    self.PIPES_INTERVAL = 3

    self.pipesElapsed = 0

    self.bird = Bird(self)
    self.pipes = []

    self.gameIsOver = False

    self.points = 0

    self.metadata = {
      "agent_type": self.AGENT_TYPE,
      "game_width": self.GAME_WIDTH,
      "game_height": self.GAME_HEIGHT,
      "bird_x_ratio": self.BIRD_X_RATIO,
      "bird_width": self.BIRD_WIDTH,
      "bird_height": self.BIRD_HEIGHT,
      "gravity": self.GRAVITY,
      "pipe_gap": self.PIPE_GAP,
      "pipe_width": self.PIPE_WIDTH,
      "pipe_velocity": self.PIPE_VELOCITY,
      "pipes_interval": self.PIPES_INTERVAL,
    }

  def update(self, dt):
    for pipePair in self.pipes:
      if self.areColliding(self.bird.hitbox, pipePair.topHitbox) or \
      self.areColliding(self.bird.hitbox, pipePair.botHitbox):
        self.gameIsOver = True
        return

      if not(pipePair.passed) and self.bird.x > pipePair.x:
        self.points += 1
        pipePair.passed = True

      pipePair.update(dt)
    self.bird.update(dt)

  def gameLoop(self, dt):
    if not self.gameIsOver:
      self.update(dt)

    if self.gameIsOver:
      if self.episodeId + 1 >= MAX_EPISODES:
        return

      self.gameIsOver = False
      self.reset()
    else:
      if self.record and len(self.frameStates) < MAX_FRAMES:
        self.frameStates.append({
          'episode_id': self.episodeId,
          'frame_id': self.frameId,
          'bird_y': self.bird.y,
          'bird_velocity': self.bird.velocity,
          'action': 1 if self.bird.isFlapping else 0,
          'pipe_pair_number': len(self.pipes),
          'pipes': (
            base64.b64encode(
              json.dumps([
                {
                  "x": pipe_pair.x,
                  "top_y": pipe_pair.topY,
                  "bot_y": pipe_pair.botY,
                  "passed": pipe_pair.passed
                }
                for pipe_pair in self.pipes
              ]).encode("utf-8")
            ).decode("utf-8")
            if len(self.pipes) > 0 else 'null'
          ),
          'score': self.points,
          'game_is_over': 1 if self.gameIsOver else 0,
        })

      self.pipesElapsed += dt

      if self.pipesElapsed >= self.PIPES_INTERVAL:
        self.addPipePair()
        self.pipesElapsed = 0

      self.bird.isFlapping = False;
      self.frameId += 1

  def reset(self):
    self.episodeId += 1
    self.frameId = 0

    self.pipesElapsed = 0

    self.bird.reset()

    self.pipes = []

    self.points = 0

  def addPipePair(self):
    pipePair = Pipes(self)
    self.pipes.append(pipePair)
  def removePipePair(self, pipePair):
    if pipePair in self.pipes:
      self.pipes.remove(pipePair)

  def areColliding(self, bird, pipe):
    return bird['x'] < pipe['x'] + pipe['width'] and \
      bird['x'] + bird['width'] > pipe['x'] and \
      bird['y'] < pipe['y'] + pipe['height'] and \
      bird['y'] + bird['height'] > pipe['y'] - 30

## Genetic algorithm logic

### GA-specific constants

In [7]:
POP_SIZE = 20
GENERATIONS = 30
ELITE_K = 5

### GA helper functions

In [8]:
def random_genome():
    # Simplified genome - only 3 weights are enough
    return {
      "y_diff_weight": random.uniform(-1.0, 1.0),
      "vel_weight": random.uniform(-1.0, 1.0),
      "bias": random.uniform(-1.0, 1.0) # Bias is like threshold
    }

def mutate(genome):
    g = genome.copy()
    if random.random() < 0.5: # No reason to always mutate everything
      g["y_diff_weight"] += random.uniform(-0.2, 0.2)
    if random.random() < 0.5:
      g["vel_weight"] += random.uniform(-0.2, 0.2)
    if random.random() < 0.5:
      g["bias"] += random.uniform(-0.1, 0.1)
    return g

def fitness(game):
    # Rewarding survival and proximity to center of gap at the moment of death

    dist_to_center = 0
    pipe = next((p for p in game.pipes if p.x + game.PIPE_WIDTH > game.bird.x), None)
    if pipe:
      # Calculate by how much the center was missed (0 to Height)
      dist_to_center = abs(game.bird.y - (pipe.topY + pipe.botY) / 2)

    # Smaller dist_to_center = better fitness
    fitness_score = game.frameId + (game.points * 1000) - dist_to_center

    return max(0, fitness_score)

### GA agent class

In [9]:
class GAAgent:
    def __init__(self, genome):
      self.genome = genome

    def act(self, game):
      target_y = game.GAME_HEIGHT / 2
      pipe = next((p for p in game.pipes if p.x + game.PIPE_WIDTH > game.bird.x), None)

      if pipe:
          target_y = (pipe.topY + pipe.botY) / 2

      # Positive if bird is ABOVE gap (needs to fall)
      # Negative if bird is below gap (needs to flap)
      y_diff = (game.bird.y - target_y) / game.GAME_HEIGHT
      velocity = game.bird.velocity / 500 # Normalization

      decision = (
          self.genome["y_diff_weight"] * y_diff +
          self.genome["vel_weight"] * velocity +
          self.genome["bias"]
      )

      return 1 if decision > 0 else 0

### Run GA episode function

In [15]:
def run_episode(genome):
  game = Game(GAME_WIDTH, GAME_HEIGHT, record=False)
  game.AGENT_TYPE = "ga"

  agent = GAAgent(genome)

  while not(game.gameIsOver) and game.frameId < MAX_FRAMES:
    action = agent.act(game)

    if action == 1 and game.bird.flapCooldown <= 0:
      game.bird.isFlapping = True
      game.bird.flapWings()
      game.bird.flapCooldown = 0.15

    game.gameLoop(FIXED_DT)

  return fitness(game)

### Main GA loop

In [23]:
ga_log = []

population = [random_genome() for _ in range(POP_SIZE)]

global_best = float("-inf")
best_genome = None
best_genome_gen = None

for gen in range(GENERATIONS):
  def run_episode_avg(genome, n=1):
    return sum(run_episode(genome) for _ in range(n)) / n
  scores = [run_episode_avg(g, n=3) for g in population]

  gen_best = max(scores)
  gen_mean = sum(scores) / len(scores)
  gen_min = min(scores)

  ga_log.append({
    "generation": gen,
    "best": gen_best,
    "mean": gen_mean,
    "min": gen_min,
  })

  print(f"Gen {gen}: best = {gen_best}")

  if gen_best > global_best:
    global_best = gen_best
    best_idx = scores.index(gen_best)
    best_genome = population[best_idx].copy()
    best_genome_gen = gen

  elite_idx = sorted(
    range(len(scores)),
    key=lambda i: scores[i],
    reverse=True
  )[:ELITE_K]

  elites = [population[i] for i in elite_idx]

  population = elites + [
    mutate(random.choice(elites))
    for _ in range(POP_SIZE - ELITE_K)
  ]

Gen 0: best = 131.10756298160646
Gen 1: best = 176.9091709768334
Gen 2: best = 271.8914774493178
Gen 3: best = 1372.2890797140626
Gen 4: best = 1097.4776395592678
Gen 5: best = 2606.623125020293
Gen 6: best = 2629.5940564948164
Gen 7: best = 6163.450869410172
Gen 8: best = 3841.8510497983243
Gen 9: best = 6637.798249526903
Gen 10: best = 5810.08825196923
Gen 11: best = 5405.229041732639
Gen 12: best = 4508.267906914803
Gen 13: best = 5758.703571860703
Gen 14: best = 5750.681089298429
Gen 15: best = 7286.421527544196
Gen 16: best = 9681.252440077378
Gen 17: best = 7286.051227805529
Gen 18: best = 8551.411827950695
Gen 19: best = 6240.756251616975
Gen 20: best = 7381.522107563672
Gen 21: best = 13684.924213604187
Gen 22: best = 43602.2903013746
Gen 23: best = 13628.651522196924
Gen 24: best = 16360.17947615763
Gen 25: best = 194925.91703100284
Gen 26: best = 194975.89718571244
Gen 27: best = 194984.42669898304
Gen 28: best = 194986.62120806938
Gen 29: best = 194985.6053382889


### Replay best genome

In [37]:
print("Replaying GA agent with genome:", best_genome)

game = Game(GAME_WIDTH, GAME_HEIGHT, record=True)
game.AGENT_TYPE = "ga"

agent = GAAgent(best_genome)

accumulator = 0.0
last_time = time.perf_counter()

while not(game.gameIsOver) and game.frameId < MAX_FRAMES:
    now = time.perf_counter()
    dt = now - last_time
    last_time = now

    accumulator += dt

    while accumulator >= FIXED_DT:
        action = agent.act(game)

        if action == 1 and game.bird.flapCooldown <= 0:
            game.bird.isFlapping = True
            game.bird.flapWings()
            game.bird.flapCooldown = 0.15

        game.gameLoop(FIXED_DT)
        accumulator -= FIXED_DT

        if game.frameId % 1500 == 0:
          print(
            f"Frame {game.frameId} | "
            f"Y={game.bird.y:.1f} | "
            f"Vel={game.bird.velocity:.1f} | "
            f"Score={game.points}"
          )

print("Replay finished")
print("Total frames:", len(game.frameStates))
print("Final score:", game.points)


Replaying GA agent with genome: {'y_diff_weight': 1.6080582416293712, 'vel_weight': 0.07046642950331658, 'bias': -0.08104066158823373}
Frame 1500 | Y=410.4 | Vel=-259.9 | Score=7
Frame 3000 | Y=507.4 | Vel=-19.4 | Score=15
Frame 4500 | Y=510.2 | Vel=321.2 | Score=23
Frame 6000 | Y=446.5 | Vel=-59.5 | Score=32
Frame 7500 | Y=338.0 | Vel=-149.7 | Score=40
Frame 9000 | Y=567.0 | Vel=251.1 | Score=48
Frame 10500 | Y=559.5 | Vel=-169.7 | Score=57
Frame 12000 | Y=364.0 | Vel=-49.5 | Score=65
Frame 13500 | Y=280.5 | Vel=-219.8 | Score=73
Frame 15000 | Y=54.0 | Vel=60.7 | Score=82
Frame 16500 | Y=466.3 | Vel=130.9 | Score=90
Frame 18000 | Y=419.4 | Vel=-119.6 | Score=98
Frame 19500 | Y=157.5 | Vel=70.7 | Score=107
Frame 21000 | Y=229.0 | Vel=-219.8 | Score=115
Frame 22500 | Y=314.8 | Vel=-129.7 | Score=123
Frame 24000 | Y=425.5 | Vel=-169.7 | Score=132
Frame 25500 | Y=123.3 | Vel=50.7 | Score=140
Frame 27000 | Y=165.9 | Vel=-109.6 | Score=148
Frame 28500 | Y=393.3 | Vel=-269.9 | Score=157
Fram

## Export game

### Export frames.csv

In [38]:
frames_headers = list(game.frameStates[0].keys())
frames_csv = "frames.csv"

with open(frames_csv, "w", newline="", encoding="utf-8") as f:
  writer = csv.writer(f)
  writer.writerow(frames_headers)
  writer.writerows(
    [[frame[h] for h in frames_headers] for frame in game.frameStates]
  )

### Export GA metadata.csv

In [39]:
game.metadata.update({
  "agent_type": "ga",
  "ga_population_size": POP_SIZE,
  "ga_generations": GENERATIONS,
  "best_genome_generation": best_genome_gen,
  "best_genome": json.dumps(best_genome),
  "max_frames": MAX_FRAMES,
})

metadata_csv = "metadata.csv"
meta_keys = list(game.metadata.keys())
meta_values = list(game.metadata.values())

with open(metadata_csv, "w", newline="", encoding="utf-8") as f:
  writer = csv.writer(f)
  writer.writerow(meta_keys)
  writer.writerow(meta_values)

### Zip & download game

In [40]:
now = datetime.datetime.now(ZoneInfo("Europe/Zagreb"))
timestamp = now.strftime("%Y%m%d_%H%M%S")
zip_filename = f"flappy_bird_ga_replay_{timestamp}.zip"

with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
  zipf.write(frames_csv)
  zipf.write(metadata_csv)

files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>